In [1]:
import mysql.connector as connector
import os
from mysql.connector import Error

In [2]:
# Establish connection between Python and MySQL database via connector API
connection=connector.connect(user="root", password=os.environ["MYSQL_PASSWORD"], port=33061)

# Create cursor object to communicate with entire MySQL database
cursor = connection.cursor()

cursor.execute("show databases")
for database in cursor:
    print(database[0])

CVD
Mangata_Gallo
STAFF_LOCATIONS
coffee_shop_products
db_Exercise
db_hr
db_learner
db_little_lemon
db_meta
db_mysqladmin
db_sports_club
db_talently
db_world
information_schema
little_lemon
little_lemon_db
lucky_Shrub
mysql
performance_schema
sakila
sys


In [3]:
#cursor = connection.cursor()
cursor.execute("USE lucky_Shrub")

In [4]:
cursor.execute("SHOW TABLES;")
for table in cursor:
    print(table)

('Bookings',)
('Customers',)
('Starters',)
('orders',)


# Functions

Wrap code within the body of a function or procedure for repeated use

`Functions`-> Accept only input parameters  
`Stored Procedure` -> Accept both input and output parameters


### Variables and Parameters
`Variable` -> Used to pass values between SQL statements, or between a procedure and a SQL statement

In Stored Procedure -> Variables are created inside or outside of a stored prodedure
In Select Statement -> Variables are created inside or outside of a select statement

`@variable_name` = value;

In Stored Procedure: Use the `SET` command: Assigns a value to a variable within a stored procedure  
`SET @variable_name` = value;  
SET @order_id = 3;  
SELECT * FROM Orders WHERE OrderID = @order_id


Using the DECLARE command variables  
`DECLARE variable_name DATATYPE DEFAULT VALUE`    
DECLARE minimum_order_cost DECIMAL(5, 2) DEFAULT 0;



In Select Statement: 
```sql
SELECT @variable_name := value;  
SELECT @max_order := MAX(Cost) FROM Orders; 
SELECT @max_order  
SELECT function() INTO variable_name FROM table_name;  
SELECT AVG(COST)  INTO @average_cost FROM Orders;  
```


`Parameters` -> Pass arguments, or values, to a function or procedure from the outside  
Types of Parameters: IN, OUT, INOUT

**IN parameter syntax** 
```sql
CREATE PROCEDURE procedure_name(IN logic(value1, value2)) SELECT logic;  
CREATE PROCEDURE CalculateTax(IN Salary DECIMAL(10, 2)) SELECT Salary * 0.2 AS TAX; 

CALL CalculateTax(10000)  
```

**OUT parameter syntax**  
```sql
CREATE PROCEDURE GetLowestCost (OUT LowestCost DECIMAL(6, 2))  
SELECT MIN(Cost) INTO LowestCost FROM Orders  
    
CALL GetLowestCost(@order_lowest_cost)  
SELECT @order_lowest_cost  
```




**INOUT parameter syntax**   
```sql
CREATE PROCEDURE SquareAnumber(INOUT aNumber INT)      
BEGIN       
      set aNumber = aNumber * aNumber     
END

SET @x_number = 5;

CALL SquareAnumber(@x_number);

SELECT @x_number;
```



**CREATING A FUNCTION**
```sql
CREATE FUNCTION function_name()
RETURNS datatype DETERMINISTIC
RETURN
```


In [5]:
create_function = """
CREATE FUNCTION GetTotalCost(Cost DECIMAL(5,2))
RETURNS DECIMAL(5, 2) DETERMINISTIC
BEGIN
IF (Cost >= 100 AND Cost < 500) THEN SET Cost = Cost - (Cost * 0.1);
ELSEIF (Cost >= 500) THEN SET Cost = Cost - (Cost * 0.2);
END IF;
RETURN (Cost);
END
"""
cursor.execute("DROP FUNCTION IF EXISTS GetTotalCost")

cursor.execute(create_function)

cursor.execute("SELECT GetTotalCost(500)")
result = cursor.fetchone()
print(result[0])

400.00


In [6]:
cursor.execute("DROP FUNCTION GetTotalCost")

try:
    cursor.execute("SELECT GetTotalCost(500)")
    result = cursor.fetchone()
    print(result[0])
except Error as err:
    print(err.msg)

FUNCTION lucky_Shrub.GetTotalCost does not exist


In [7]:
create_function = """
CREATE FUNCTION FindCost(order_id INT)
RETURNS DECIMAL(5, 2) DETERMINISTIC
RETURN (SELECT Cost FROM orders WHERE OrderID = order_id);
"""
cursor.execute("DROP FUNCTION IF EXISTS FindCost")

In [8]:
cursor.execute(create_function)

In [9]:
cursor.execute("SELECT FindCost(5)")
result = cursor.fetchone()
print(result[0])

450.00


# Prodecure 

In [10]:
stored_procedure_query = """
CREATE Procedure GetDiscount(OrderIDInput INT) 
BEGIN 
DECLARE cost_after_discount DECIMAL(7,2); 
DECLARE current_cost DECIMAL(7,2); 
DECLARE order_quantity INT; 
SELECT Quantity INTO order_quantity FROM orders WHERE OrderID = OrderIDInput; 
SELECT Cost INTO current_cost FROM orders WHERE OrderID = OrderIDInput; 
IF order_quantity >= 20 THEN
SET cost_after_discount = current_cost - (current_cost * 0.2);              
ELSEIF order_quantity >= 10 THEN
SET cost_after_discount = current_cost - (current_cost * 0.1); 
ELSE SET cost_after_discount = current_cost;
END IF;
SELECT cost_after_discount; 
END
"""
drop_procedure_query = """
DROP PROCEDURE IF EXISTS GetDiscount
"""
cursor.execute(drop_procedure_query)
connection.commit()

cursor.execute(stored_procedure_query)
cursor.callproc("GetDiscount", (5,))

(5,)

In [11]:
results=next(cursor.stored_results())
dataset = results.fetchall()
for data in dataset:
    print(data)

(Decimal('405.00'),)


# Triggers
A set of actions available in the form of a stored program invoked when an event occurs e.g INSERT, UPDATE, DELETE. MySQL only supports Row-level Triggers

```sql
CREATE TRIGGER trigger_name
TRIGGER TYPE
ON table_name FOR EACH ROW
BEGIN
    statement_one
    statement_two
END


DROP TRIGGER IF EXISTS schema_name.trigger_name
```

Keeping a log of records: useful for maintaining audit trails  
Alternative to constraints: Maintain data integrity and run tasks


**Types of Triggers**
- Row-level: Trigger invoked for each row
- Statement-level: Trigger invoked for each statement

```sql
CREATE TRIGGER AfterDeleteOrder
AFTER DELETE
ON Orders FOR EACH ROW
INSERT INTO Audits
VALUES('AFTER', CONCAT('Order', OLD.OrderID, 'was deleted at', CURRENT_TIME(), 'on', CURRENT_DATE()), 'DELETE')
```


```sql
CREATE TRIGGER OrderQtyCheck
BEFORE INSERT 
ON Orders FOR EACH ROW
BEGIN
    IF NEW.Quantity < 0 THEN SET NEW.QUANTITY = 0;
    END IF;
END
```


**Scheduled Events***
A task that takes place at a specific time according to a schedule

```sql
CREATE EVENT IF NOT EXISTS event_name
ON SCHEDULE schedule_logic
DO 
Event_body
```


```sql
CREATE EVENT IF NOT EXISTS event_name
ON SCHEDULE AT CURRENT_TIMESTAMP [+ INTERVAL]
DO 
Event_body
```

```sql
CREATE EVENT GenerateRevenueReport
ON SCHEDULE AT CURRENT_TIMESTAMP + INTERVAL 12 HOUR
DO
    BEGIN
        INSERT INTO ReportData (OrderID, ClientID, ProductID, Quantity, Cost, Date)
        SELECT * FROM Orders WHERE Date BETWEEN '2022-08-01' AND '2022-08-31';
    END
```

In [12]:
cursor.execute("create database if not exists meta_db;")
cursor.execute("show databases;")
for database in cursor:
    print(database[0])

CVD
Mangata_Gallo
STAFF_LOCATIONS
coffee_shop_products
db_Exercise
db_hr
db_learner
db_little_lemon
db_meta
db_mysqladmin
db_sports_club
db_talently
db_world
information_schema
little_lemon
little_lemon_db
lucky_Shrub
meta_db
mysql
performance_schema
sakila
sys


In [13]:
cursor.execute("USE meta_db;")

In [14]:
cursor.execute("SHOW TABLES;")
for table in cursor:
    print(table)

In [15]:
create_product = """CREATE TABLE Products (ProductID VARCHAR(10), ProductName VARCHAR(100), BuyPrice DECIMAL(6,2), SellPrice DECIMAL(6,2), NumberOfItems INT);"""
cursor.execute(create_product)

In [16]:
insert_query = """ INSERT INTO Products (ProductID, ProductName, BuyPrice, SellPrice, NumberOfItems)
VALUES ("P1", "Artificial grass bags ", 40, 50, 100),  
("P2", "Wood panels", 15, 20, 250),  
("P3", "Patio slates",35, 40, 60),  
("P4", "Sycamore trees ", 7, 10, 50),  
("P5", "Trees and Shrubs", 35, 50, 75),  
("P6", "Water fountain", 65, 80, 15);
"""

cursor.execute(insert_query)
connection.commit()

In [17]:
cursor.execute("SELECT * FROM Products;")
results = cursor.fetchall()
for result in results:
    print(result)

('P1', 'Artificial grass bags ', Decimal('40.00'), Decimal('50.00'), 100)
('P2', 'Wood panels', Decimal('15.00'), Decimal('20.00'), 250)
('P3', 'Patio slates', Decimal('35.00'), Decimal('40.00'), 60)
('P4', 'Sycamore trees ', Decimal('7.00'), Decimal('10.00'), 50)
('P5', 'Trees and Shrubs', Decimal('35.00'), Decimal('50.00'), 75)
('P6', 'Water fountain', Decimal('65.00'), Decimal('80.00'), 15)


In [18]:
create_notifications = """CREATE TABLE IF NOT EXISTS Notifications (NotificationID INT AUTO_INCREMENT, Notification VARCHAR(255), DateTime TIMESTAMP NOT NULL, PRIMARY KEY(NotificationID));"""
cursor.execute(create_notifications)
cursor.execute("SHOW TABLES;")
for table in cursor:
    print(table)

('Notifications',)
('Products',)


### Task 1:


<div class="alert alert-info" style="margin-top: 15px">
<p>Create an INSERT trigger called <strong>ProductSellPriceInsertCheck</strong>. This trigger must check if the SellPrice of the product is less than the BuyPrice after a new product is inserted in the Products table. If this occurs, then a notification must be added to the Notifications table to inform the sales department. The sales department can then ensure that the incorrect values were not inserted by mistake.</p>
<p>The notification message should be in the following format: A SellPrice less than the BuyPrice was inserted for ProductID + ProductID</p>
</div>





In [19]:
drop_query = """DROP TRIGGER IF EXISTS meta_db.ProductSellPriceInsertCheck"""
cursor.execute(drop_query)
connection.commit()



create_trigger_query = """
CREATE TRIGGER ProductSellPriceInsertCheck
AFTER INSERT
ON Products FOR EACH ROW
BEGIN
    IF New.SellPrice < New.BuyPrice THEN
    INSERT INTO Notifications(Notification, DateTime)
    VALUES(CONCAT('A SellPrice less than the BuyPrice was inserted for ProductID ',  New.ProductID), NOW());
    END IF;
END
"""
cursor.execute(create_trigger_query)

In [20]:
cursor.execute("SHOW TABLES;")
for table in cursor:
    print(table)

('Notifications',)
('Products',)


In [21]:
insert_query = """INSERT INTO meta_db.Products(ProductID, ProductName, BuyPrice, SellPrice, NumberOfItems) VALUES ("P7", "Product p7", 45, 40, 100);"""
cursor.execute(insert_query)
connection.commit()

In [22]:
cursor.execute("SELECT * FROM Products;")
results = cursor.fetchall()
for result in results:
    print(result)

('P1', 'Artificial grass bags ', Decimal('40.00'), Decimal('50.00'), 100)
('P2', 'Wood panels', Decimal('15.00'), Decimal('20.00'), 250)
('P3', 'Patio slates', Decimal('35.00'), Decimal('40.00'), 60)
('P4', 'Sycamore trees ', Decimal('7.00'), Decimal('10.00'), 50)
('P5', 'Trees and Shrubs', Decimal('35.00'), Decimal('50.00'), 75)
('P6', 'Water fountain', Decimal('65.00'), Decimal('80.00'), 15)
('P7', 'Product p7', Decimal('45.00'), Decimal('40.00'), 100)


In [23]:
cursor.execute("SELECT * FROM Notifications;")
results = cursor.fetchall()
for result in results:
    print(result)

(1, 'A SellPrice less than the BuyPrice was inserted for ProductID P7', datetime.datetime(2024, 10, 5, 3, 51, 53))


## Task 2:

<div class="alert alert-info" style="margin-top: 15px">
<p>Create an UPDATE trigger called <strong>ProductSellPriceUpdateCheck</strong>. This trigger must check that products are not updated with a SellPrice that is less than or equal to the BuyPrice. If this occurs, add a notification to the Notifications table for the sales department so they can ensure that product prices were not updated with the incorrect values. This trigger sends a notification to the Notifications table that warns the sales department of the issue.</p>
<p>The notification message should be in the following format: ProductID + was updated with a SellPrice of  + SellPrice + which is the same or less than the BuyPrice</p>
</div>

In [24]:
drop_query = """DROP TRIGGER IF EXISTS ProductSellPriceUpdateCheck"""
cursor.execute(drop_query)
connection.commit()


create_trigger_query = """
CREATE TRIGGER ProductSellPriceUpdateCheck
AFTER UPDATE
ON Products FOR EACH ROW
BEGIN
    IF New.SellPrice <= New.BuyPrice THEN
    INSERT INTO Notifications(Notification, DateTime)
    VALUES(CONCAT(New.ProductID, ' was updated with a SellPrice of ',  New.SellPrice, ' which is the same or less than the BuyPrice'), NOW());
    END IF;
END
"""
cursor.execute(create_trigger_query)

In [25]:
cursor.execute("SELECT * FROM Products;")
results = cursor.fetchall()
for result in results:
    print(result)

('P1', 'Artificial grass bags ', Decimal('40.00'), Decimal('50.00'), 100)
('P2', 'Wood panels', Decimal('15.00'), Decimal('20.00'), 250)
('P3', 'Patio slates', Decimal('35.00'), Decimal('40.00'), 60)
('P4', 'Sycamore trees ', Decimal('7.00'), Decimal('10.00'), 50)
('P5', 'Trees and Shrubs', Decimal('35.00'), Decimal('50.00'), 75)
('P6', 'Water fountain', Decimal('65.00'), Decimal('80.00'), 15)
('P7', 'Product p7', Decimal('45.00'), Decimal('40.00'), 100)


In [26]:
update_query = """UPDATE Products SET SellPrice = 60 WHERE ProductID = 'P6';"""
cursor.execute(update_query)
connection.commit()

In [27]:
cursor.execute("SELECT * FROM Products;")
results = cursor.fetchall()
for result in results:
    print(result)

('P1', 'Artificial grass bags ', Decimal('40.00'), Decimal('50.00'), 100)
('P2', 'Wood panels', Decimal('15.00'), Decimal('20.00'), 250)
('P3', 'Patio slates', Decimal('35.00'), Decimal('40.00'), 60)
('P4', 'Sycamore trees ', Decimal('7.00'), Decimal('10.00'), 50)
('P5', 'Trees and Shrubs', Decimal('35.00'), Decimal('50.00'), 75)
('P6', 'Water fountain', Decimal('65.00'), Decimal('60.00'), 15)
('P7', 'Product p7', Decimal('45.00'), Decimal('40.00'), 100)


In [28]:
cursor.execute("SELECT * FROM Notifications;")
results = cursor.fetchall()
for result in results:
    print(result)

(1, 'A SellPrice less than the BuyPrice was inserted for ProductID P7', datetime.datetime(2024, 10, 5, 3, 51, 53))
(2, 'P6 was updated with a SellPrice of 60.00 which is the same or less than the BuyPrice', datetime.datetime(2024, 10, 5, 3, 51, 54))


## Task 3:

<div class="alert alert-info" style="margin-top: 15px">
<p>Create a DELETE trigger called <strong>NotifyProductDelete</strong>. This trigger must insert a notification in the Notifications table for the sales department after a product has been deleted from the Products table.</p>
<p>The notification message should be in the following format: The product with a ProductID  + ProductID + was deleted</p>
</div>

In [29]:
drop_query = """DROP TRIGGER IF EXISTS NotifyProductDelete"""
cursor.execute(drop_query)
connection.commit()


create_trigger_query = """CREATE TRIGGER NotifyProductDelete
AFTER DELETE
ON Products FOR EACH ROW
BEGIN
    INSERT INTO Notifications(Notification, DateTime)
    VALUES(CONCAT('The product with a ProductID ', OLD.ProductID,' was deleted'), NOW());
END
"""
cursor.execute(create_trigger_query)

In [30]:
delete_query = """DELETE FROM Products WHERE ProductID = 'P7';"""
cursor.execute(delete_query)
connection.commit()

In [31]:
cursor.execute("SELECT * FROM Products;")
results = cursor.fetchall()
for result in results:
    print(result)

('P1', 'Artificial grass bags ', Decimal('40.00'), Decimal('50.00'), 100)
('P2', 'Wood panels', Decimal('15.00'), Decimal('20.00'), 250)
('P3', 'Patio slates', Decimal('35.00'), Decimal('40.00'), 60)
('P4', 'Sycamore trees ', Decimal('7.00'), Decimal('10.00'), 50)
('P5', 'Trees and Shrubs', Decimal('35.00'), Decimal('50.00'), 75)
('P6', 'Water fountain', Decimal('65.00'), Decimal('60.00'), 15)


In [32]:
cursor.execute("SELECT * FROM Notifications;")
results = cursor.fetchall()
for result in results:
    print(result)

(1, 'A SellPrice less than the BuyPrice was inserted for ProductID P7', datetime.datetime(2024, 10, 5, 3, 51, 53))
(2, 'P6 was updated with a SellPrice of 60.00 which is the same or less than the BuyPrice', datetime.datetime(2024, 10, 5, 3, 51, 54))
(3, 'The product with a ProductID P7 was deleted', datetime.datetime(2024, 10, 5, 3, 51, 54))


In [33]:
connection.close()